# 0. `TildeRPairNormalizer`

In [20]:
import numpy as np
from matersdk.io.publicLayer.structure import DStructure
from matersdk.io.publicLayer.neigh import StructureNeighborsDescriptor
from matersdk.feature.deepmd.se_pair import DpseTildeRPairDescriptor
from matersdk.feature.deepmd.preprocess import TildeRPairNormalizer

# 1. Custom parameters

In [11]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/feature/movement/LiSi.config"
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/feature/movement/LiSi_32.config"
scaling_matrix = [3, 3, 3]
reformat_mark = True
coords_are_cartesian = True

center_atomic_number = 3    # Li
nbr_atomic_number = 14      # Si
rcut = 6.5
rcut_smooth = 6.0

# 2. Initialize `StructureNeighbors`

In [12]:
### Step 1.
structure = DStructure.from_file(
                file_format="pwmat",
                file_path=atom_config_path)

new_structure = DStructure.from_file(
                file_format="pwmat",
                file_path=atom_config_path)


### Step 2.
struct_nbr = StructureNeighborsDescriptor.create(
            'v1',
            structure=structure,
            rcut=rcut,
            scaling_matrix=scaling_matrix,
            reformat_mark=reformat_mark,
            coords_are_cartesian=coords_are_cartesian)

new_struct_nbr = StructureNeighborsDescriptor.create(
            'v1',
            structure=new_structure,
            rcut=rcut,
            scaling_matrix=scaling_matrix,
            reformat_mark=reformat_mark,
            coords_are_cartesian=coords_are_cartesian)


### Step 3. 
dpse_tildeR_pair = DpseTildeRPairDescriptor.create(
                'v1',
                structure_neighbors=struct_nbr,
                center_atomic_number=center_atomic_number,
                nbr_atomic_number=nbr_atomic_number,
                rcut=rcut,
                rcut_smooth=rcut_smooth)
tildeRs_array = dpse_tildeR_pair.dp_feature_pair_tildeR

new_dpse_tildeR_pair = DpseTildeRPairDescriptor.create(
                'v1',
                structure_neighbors=new_struct_nbr,
                center_atomic_number=center_atomic_number,
                nbr_atomic_number=nbr_atomic_number,
                rcut=rcut,
                rcut_smooth=rcut_smooth)
new_tildeRs_array = new_dpse_tildeR_pair.dp_feature_pair_tildeR

# 3. Initialize `DpseTildeRPairNormalizer`

In [17]:
normalizer = TildeRPairNormalizer(tildeRs_array=tildeRs_array)
print("Step 1.1. davg of Environment matrix : ", end='\n\t')
print(normalizer.davg)
print("Step 1.2. dstd of Environment matrix : ", end='\n\t')
print(normalizer.dstd)

Step 1.1. davg of Environment matrix : 
	[[0.211878 0.       0.       0.      ]]
Step 1.2. dstd of Environment matrix : 
	[[0.10693948 0.13702576 0.13702576 0.13702576]]


In [19]:
1 / normalizer.davg[0][0]

4.7196970861147705

In [24]:
print("Step 2. Using a new environment matrix, after normalize...")
print("Step 2.1. The max value of environment is : ", end="\n\t")
print(np.max(normalizer.normalize(tildeRs_array=new_tildeRs_array)))
print("Step 2.2. The min value of environment is : ", end="\n\t")
print(np.min(normalizer.normalize(tildeRs_array=new_tildeRs_array)))

Step 2. Using a new environment matrix, after normalize...
Step 2.1. The max value of environment is : 
	2.9726018812386306
Step 2.2. The min value of environment is : 
	-2.9534562464027796
